# Load Dataset

In [17]:
################################################################################
# Load dataset and split it into training and test set
################################################################################

import pandas as pd
import os
from tabulate import tabulate

dataset_name = "wustl-iiot"
sample_size = 100000

# Load dateset
df = pd.read_csv(os.getcwd() + f'/data/sample-{sample_size}-2.csv')

# Split dataset according to attack type
normal_df = df[df['Target'] == 0]
attack_df = df[df['Target'] == 1]

# Drop columns
normal_df = normal_df.drop(columns=['Target', 'Traffic'])
attack_df = attack_df.drop(columns=['Target', 'Traffic'])

# Split dataset into training and test set
normal_df_train = normal_df.sample(frac=0.8, random_state=42)
normal_df_test = normal_df.drop(normal_df_train.index)
attack_df_train = attack_df.sample(frac=0.8, random_state=42)
attack_df_test = attack_df.drop(attack_df_train.index)

# Print dataset sizes in a table
data = [
    ["Normal", normal_df.shape[0], normal_df_train.shape[0], normal_df_test.shape[0]],
    ["Attack", attack_df.shape[0], attack_df_train.shape[0], attack_df_test.shape[0]]
]
print(tabulate(data, headers=["Atack type", "Total", "Train", "Test"], tablefmt="grid"))

+--------------+---------+---------+--------+
| Atack type   |   Total |   Train |   Test |
+==============+=========+=========+========+
| Normal       |   50000 |   40000 |  10000 |
+--------------+---------+---------+--------+
| Attack       |   50000 |   40000 |  10000 |
+--------------+---------+---------+--------+


# Feature Importance

In [ ]:
################################################################################
# Generate Feature Importance
################################################################################

import os
import dotenv
import time
import numpy as np
import json
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

dotenv.load_dotenv(os.getcwd() + '/../.env')

template = """
You are provided with network data entries categorized as either normal or attack, along with their corresponding feature names.
Carefully analyze the differences between normal and attack entries by comparing corresponding fields.
Output top 10 important features that can be used to filter an entry as either normal or attack.
Output only in the Python list structure.

Normal Entries:
```{normal_entries}```

Attack Entries:
```{attack_entries}```

Example output:
['feature1', 'feature2', 'feature3', ..., 'feature10']
"""

prompt = PromptTemplate(template=template, input_variables=["normal_entries", "attack_entries"])
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
model_name = "gpt-4o"
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.0)
# model_name = "gemini-1.5-pro"
# llm = ChatAnthropic(model='claude-3-opus-20240229')
# model_name = "claude-3-opus-20240229"
chain = prompt | llm
train_set_size = sample_size
embeddings = HuggingFaceEmbeddings()
vector_store = Chroma(
    collection_name=dataset_name,
    embedding_function=embeddings, 
    persist_directory=f"./vector-stores/chroma-db-{train_set_size}-2")

normal_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'normal'})['embeddings']
normal_mean_vector = np.mean(normal_vectors, axis=0).tolist()
normal_documents = vector_store._collection.query(query_embeddings=[normal_mean_vector], n_results=10)['documents'][0]

attack_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'attack'})['embeddings']
attack_mean_vector = np.mean(attack_vectors, axis=0).tolist()
attack_documents = vector_store._collection.query(query_embeddings=[attack_mean_vector], n_results=10)['documents'][0]

normal_entries = {}
for i, feature_name in enumerate(normal_df_train.columns.to_list()):
    normal_entries[feature_name] = [json.loads(doc)[i] for doc in normal_documents]

attack_entries = {}
for i, feature_name in enumerate(attack_df_train.columns.to_list()):
    attack_entries[feature_name] = [json.loads(doc)[i] for doc in attack_documents]

completions = []
for i in range(10):
    completion = chain.invoke({
        "normal_entries": json.dumps(normal_entries),
        "attack_entries": json.dumps(attack_entries)
    })
    completions.append(completion.content)
    print(completion.content)
    time.sleep(10)

with open(f"results/feature-importance-{sample_size}-llm-{model_name}.txt", "a") as f:
    f.write("\n".join(completions))

# Prediction

In [5]:
################################################################################
# Generate Rules with transposed data
################################################################################

import os
import dotenv
import json
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_anthropic import ChatAnthropic
from langchain_chroma import Chroma
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
import numpy as np
import uuid
import tiktoken     # https://github.com/openai/tiktoken

dotenv.load_dotenv(os.getcwd() + '/../.env')

template = """
You are provided with network data entries categorized as either normal or attack, along with their corresponding feature names.
Carefully analyze the differences between normal and attack entries by comparing corresponding fields.
Generate 5 simple and deterministic rules for top 5 important features to filter an entry as either normal or attack. 
Output only in the JSON format with the structure: 
{{'feature1': 'rule', 'feature2': 'rule', ..., 'feature5': 'rule'}}.

Normal Entries:
```{normal_entries}```

Attack Entries:
```{attack_entries}```
"""
prompt = PromptTemplate(template=template, input_variables=["normal_entries", "attack_entries"])
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
model_name = "gpt-4o"
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.0)
# model_name = "gemini-1.5-pro"
# llm = ChatAnthropic(model='claude-3-opus-20240229')
# model_name = "claude-3-opus-20240229"
chain = prompt | llm
train_set_size = sample_size
embeddings = HuggingFaceEmbeddings()
vector_store = Chroma(
    collection_name=dataset_name,
    embedding_function=embeddings, 
    persist_directory=f"./vector-stores/chroma-db-{train_set_size}-2")

normal_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'normal'})['embeddings']
normal_mean_vector = np.mean(normal_vectors, axis=0).tolist()
normal_documents = vector_store._collection.query(query_embeddings=[normal_mean_vector], n_results=10)['documents'][0]

attack_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'attack'})['embeddings']
attack_mean_vector = np.mean(attack_vectors, axis=0).tolist()
attack_documents = vector_store._collection.query(query_embeddings=[attack_mean_vector], n_results=10)['documents'][0]

normal_entries = {}
for i, feature_name in enumerate(normal_df_train.columns.to_list()):
    normal_entries[feature_name] = [json.loads(doc.replace("'", "\""))[i] for doc in normal_documents]

attack_entries = {}
for i, feature_name in enumerate(attack_df_train.columns.to_list()):
    attack_entries[feature_name] = [json.loads(doc.replace("'", "\""))[i] for doc in attack_documents]

# prompt_text = prompt.invoke({
#     "normal_entries": json.dumps(normal_entries),
#     "attack_entries": json.dumps(attack_entries)
# }).text

# print(prompt_text)

completion = chain.invoke({
    "normal_entries": json.dumps(normal_entries),
    "attack_entries": json.dumps(attack_entries)
})

print(completion.content)

id = str(uuid.uuid4())
with open(f"results/llm/generated-rules-{sample_size}-llm-{model_name}.txt", "a") as f:
    f.write(f"{id}\n")
    f.write(f"{completion.content}\n")

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
num_tokens_prompt = len(encoding.encode(prompt.invoke({"normal_entries": json.dumps(normal_entries),"attack_entries": json.dumps(attack_entries)}).text))
num_tokens_completion = len(encoding.encode(str(completion.content)))

print(f"Prompt tokens: {num_tokens_prompt}")
print(f"Completion tokens: {num_tokens_completion}")
print(f"Total tokens: {num_tokens_prompt + num_tokens_completion}")
print(f"Percentage of tokens used: {(num_tokens_prompt + num_tokens_completion) / 128000}")

c:\Users\S4025371\OneDrive - RMIT University\Repositories\iot-llm\.conda\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


```json
{
    "SrcAddr": "If SrcAddr is 192.168.0.20, then Normal; otherwise, Attack",
    "Dport": "If Dport is 502, then Normal; otherwise, Attack",
    "SrcPkts": "If SrcPkts is 10, then Normal; otherwise, Attack",
    "DstPkts": "If DstPkts is 8, then Normal; otherwise, Attack",
    "TotBytes": "If TotBytes is 1152, then Normal; otherwise, Attack"
}
```
Prompt tokens: 5289
Completion tokens: 118
Total tokens: 5407
Percentage of tokens used: 0.0422421875


In [35]:
################################################################################
# Evaluate generated rules
################################################################################

from statistics import mode
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

datasets = {"normal": normal_df_test, "attack": attack_df_test}
y_pred = []
y_true = []
for attack_type, dataset in datasets.items():
    test_set_size = dataset.shape[0]
    for i in tqdm(range(test_set_size), ncols=100, desc=f"Predicting {attack_type} entries..."):
        predicted_attack_types = []
        predicted_attack_types.append("normal" if dataset.iloc[i]['SrcAddr'] == "192.168.0.20" else "attack")
        predicted_attack_types.append("normal" if dataset.iloc[i]['Dport'] == 502 else "attack")
        predicted_attack_types.append("normal" if dataset.iloc[i]['SrcPkts'] == 10 else "attack")
        predicted_attack_types.append("normal" if dataset.iloc[i]['DstPkts'] == 8 else "attack")
        predicted_attack_types.append("normal" if dataset.iloc[i]['TotBytes'] == 1152 else "attack")
        y_true.append(attack_type)
        y_pred.append(mode(predicted_attack_types))

c_report = classification_report(y_true, y_pred, digits=4)
c_matrix = confusion_matrix(y_true, y_pred)

with open(f"results/llm/result-llm-{sample_size}-2.txt", "a") as f:
    f.write(f"{id}\n")
    f.write(f"Classication Report\n{c_report}\n\nConfusion Matrix\n{c_matrix}\n")

print(c_report)
print(c_matrix)

Predicting attack entries...: 100%|███████████████████████████| 1000/1000 [00:00<00:00, 2987.37it/s]

              precision    recall  f1-score   support

      attack     0.8658    1.0000    0.9281      1000
      normal     1.0000    0.8450    0.9160      1000

    accuracy                         0.9225      2000
   macro avg     0.9329    0.9225    0.9220      2000
weighted avg     0.9329    0.9225    0.9220      2000

[[1000    0]
 [ 155  845]]


# Feedback Loop

In [18]:
################################################################################
# Prompt Template
################################################################################
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

system_message = ("system",
"""
You are a good data analyst.
You are provided with network data entries categorized as either normal or attack, along with their corresponding feature names.
Carefully analyze the differences between normal and attack entries by comparing corresponding fields.
Your task is to generate {k} simple and deterministic rules for top {k} important features to filter attack entries.
Supported operators are '==', '!=', '>', '<', '>=', '<='.
Generate exactly {k} rules to filter attack entries and make a tool call for each rule.
"""
)
human_message = ("user",
"""
Analyze the following network data and generate rules for the top 5 important features to filter attack entries.

Normal Entries:
```{normal_entries}```

Attack Entries:
```{attack_entries}```
"""
)

prompt = ChatPromptTemplate.from_messages([
    system_message,
    human_message,
    MessagesPlaceholder("msgs")
])

# Invoke prompt
# prompt.invoke({"k": 5, "normal_entries": normal_entries, "attack_entries": attack_entries, "msgs": []})

In [19]:
################################################################################
# Tool
################################################################################

from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import operator
from typing import Annotated
from langchain_core.tools import tool

show_progress = True
operations = {'<': operator.lt, '>': operator.gt, '==': operator.eq, '<=': operator.le, '>=': operator.ge, '!=': operator.ne}

@tool
def evaluate_rule(
    feature_name: Annotated[str, "Feature name"],
    value: Annotated[str, "Value"], 
    op: Annotated[str, "Operator"]
) -> bool:
    """Evaluate the rule and return the macro f1-score."""
    try:
        value = float(value)
    except ValueError:
        value
    datasets = {"normal": normal_df_train, "attack": attack_df_train}
    y_pred = []
    y_true = []
    if op in operations:
        for attack_type, dataset in datasets.items():
            test_set_size = dataset.shape[0]
            for i in tqdm(range(test_set_size), ncols=100, desc=f"Predicting {attack_type} entries...", disable=not show_progress):
                y_true.append(attack_type)
                y_pred.append("attack" if operations[op](dataset.iloc[i][feature_name], value) else "normal")
        c_report = classification_report(y_true, y_pred, digits=4, output_dict=True)
        return c_report['macro avg']['f1-score']
    else:
        raise ValueError(f"Unsupported operator: {op}")

# Invoke tool
# print(evaluate_rule.invoke({"feature_name": "flow_duration", "value": "1", "op": "<"}))

In [20]:
################################################################################
# LLM
################################################################################

import os
import dotenv
from langchain_openai import ChatOpenAI
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_anthropic import ChatAnthropic

dotenv.load_dotenv(os.getcwd() + '/../.env')

model_name = "gpt-4o"
llm = ChatOpenAI(model=model_name, temperature=0.1)
# model_name = "gemini-1.5-pro"
# llm = ChatGoogleGenerativeAI(model=model_name, temperature=0.0)
# model_name = "claude-3-opus-20240229"
# llm = ChatAnthropic(model=model_name, temperature=0.0)

llm_with_tool = llm.bind_tools([evaluate_rule])

In [21]:
################################################################################
# Vector Store
################################################################################

import json
import numpy as np
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings

train_set_size = sample_size
n_results = 10
embeddings = HuggingFaceEmbeddings()
vector_store = Chroma(
    collection_name=dataset_name,
    embedding_function=embeddings, 
    persist_directory=f"./vector-stores/chroma-db-{train_set_size}-2")

normal_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'normal'})['embeddings']
normal_mean_vector = np.mean(normal_vectors, axis=0).tolist()
normal_documents = vector_store._collection.query(query_embeddings=[normal_mean_vector], n_results=n_results)['documents'][0]

attack_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'attack'})['embeddings']
attack_mean_vector = np.mean(attack_vectors, axis=0).tolist()
attack_documents = vector_store._collection.query(query_embeddings=[attack_mean_vector], n_results=n_results)['documents'][0]

normal_entries_dict = {}
for i, feature_name in enumerate(normal_df_train.columns.to_list()):
    normal_entries_dict[feature_name] = [json.loads(doc.replace("'", "\""))[i] for doc in normal_documents]

attack_entries_dict = {}
for i, feature_name in enumerate(attack_df_train.columns.to_list()):
    attack_entries_dict[feature_name] = [json.loads(doc.replace("'", "\""))[i] for doc in attack_documents]

In [22]:
################################################################################
# Chain
################################################################################

from langchain_core.messages import HumanMessage

chain = prompt | llm_with_tool

n_repetitions = 5
context_window = 128000
show_progress = False

def get_initial_state():
  n = 0
  k = 5
  mean_f1s = 0
  max_f1s = 0
  n_max = 0
  token_usage = {}
  normal_entries = json.dumps(normal_entries_dict)
  attack_entries = json.dumps(attack_entries_dict)
  msgs = []
  return locals()

state = get_initial_state()
train_f1_scores = []
while state["n"] < n_repetitions:
    ai_msg = chain.invoke(state)
    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        tool_msg = evaluate_rule.invoke(tool_call)
        tool_msgs.append(tool_msg)
    state["mean_f1s"] = sum(float(msg.content) for msg in tool_msgs) / len(tool_msgs)
    human_msg = HumanMessage(f"The current mean f1-score for the generated rules is {state['mean_f1s']}. "
                             "If this mean f1-score is greater than the previous rounds, keep the better performing "
                             "rules and revise or replace only the underperforming ones (those with a score less than mean). "
                             "Otherwise, revise or replace any rules that have a score less than mean. "
                             f"Based on the feedback, generate exactly {state['k']} rules to filter attack entries and "
                             "make a tool call for each rule, ensuring that a tool call is made for every entry every time.")
    state["n"] += 1
    state["msgs"].extend([ai_msg, *tool_msgs, human_msg])
    train_f1_scores.append(state["mean_f1s"])
    state["max_f1s"] = state["mean_f1s"] if state["mean_f1s"] > state["max_f1s"] else state["max_f1s"]
    state["n_max"] = state["n"] if state["mean_f1s"] > state["max_f1s"] else state["n_max"]
    state["token_usage"] = {key: ai_msg.response_metadata["token_usage"][key] for key in ["completion_tokens", "prompt_tokens", "total_tokens"]} 
    print("Round:", state["n"], "Current mean f1-score:", state["mean_f1s"], "Token usage:", state["token_usage"])

print(train_f1_scores)

Round: 1 Current mean f1-score: 0.8655846577728055 Token usage: {'completion_tokens': 453, 'prompt_tokens': 5367, 'total_tokens': 5820}
Round: 2 Current mean f1-score: 0.8363417979066222 Token usage: {'completion_tokens': 481, 'prompt_tokens': 5996, 'total_tokens': 6477}
Round: 3 Current mean f1-score: 0.8627639312673082 Token usage: {'completion_tokens': 398, 'prompt_tokens': 6653, 'total_tokens': 7051}
Round: 4 Current mean f1-score: 0.940612908306159 Token usage: {'completion_tokens': 391, 'prompt_tokens': 7207, 'total_tokens': 7598}
Round: 5 Current mean f1-score: 0.9365228303168088 Token usage: {'completion_tokens': 394, 'prompt_tokens': 7773, 'total_tokens': 8167}
[0.8655846577728055, 0.8363417979066222, 0.8627639312673082, 0.940612908306159, 0.9365228303168088]


In [23]:
################################################################################
# Evaluate generated rules
################################################################################

from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import operator
from statistics import mode

operations = {'<': operator.lt, '>': operator.gt, '==': operator.eq, '<=': operator.le, '>=': operator.ge, '!=': operator.ne}

def evaluate_rules(tool_calls):
    datasets = {"normal": normal_df_test, "attack": attack_df_test}
    y_pred = []
    y_true = []
    for attack_type, dataset in datasets.items():
        test_set_size = dataset.shape[0]
        for i in tqdm(range(test_set_size), ncols=100, desc=f"Predicting {attack_type} entries...", disable=not show_progress):
            predicted_attack_types = []
            for tool_call in tool_calls:
                args = json.loads(tool_call["function"]["arguments"])
                op = args["op"]
                feature_name = args["feature_name"]
                value = args["value"]
                try:
                    value = float(value)
                except ValueError:
                    value
                predicted_attack_types.append("attack" if operations[op](dataset.iloc[i][feature_name], value) else "normal")
            y_true.append(attack_type)
            y_pred.append(mode(predicted_attack_types))
    c_report = classification_report(y_true, y_pred, digits=4, output_dict=True)
    c_matrix = confusion_matrix(y_true, y_pred)
    # print(c_report)
    # print(c_matrix)
    return c_report

# tool_calls = state["msgs"][-7].additional_kwargs["tool_calls"]
# for tool_call in tool_calls:
#     rule = json.loads(tool_call["function"]["arguments"])
#     print("attack if", rule["feature_name"], rule["op"], rule["value"], "else normal")

# evaluate_rules(tool_calls)

# test_f1_scores = []
# for i in range(20, 0, -1):
#     index = -7 * i
#     tool_calls = state["msgs"][index].additional_kwargs["tool_calls"]
#     for tool_call in tool_calls:
#         rule = json.loads(tool_call["function"]["arguments"])
#     test_f1_scores.append(evaluate_rules(tool_calls)['macro avg']['f1-score'])

# print(test_f1_scores)

for i in range(len(state["msgs"])):
    if state["msgs"][i].type != "ai":
        continue
    tool_calls = state["msgs"][i].additional_kwargs["tool_calls"]
    for tool_call in tool_calls:
        rule = json.loads(tool_call["function"]["arguments"])
        print("attack if", rule["feature_name"], rule["op"], rule["value"], "else normal")
    c_report = evaluate_rules(tool_calls)
    print(c_report["macro avg"]["f1-score"])

attack if SrcAddr != 192.168.0.20 else normal
attack if Dport == 80 else normal
attack if SrcPkts == 2 else normal
attack if DstPkts == 0 else normal
attack if TotBytes == 124 else normal
0.8632729612271635
attack if SrcAddr != 192.168.0.20 else normal
attack if SrcRate == 1000000 else normal
attack if DstRate == 0 else normal
attack if DstPkts == 0 else normal
attack if Rate == 1000000 else normal
0.9747964906633599
attack if SrcAddr != 192.168.0.20 else normal
attack if SrcLoad == 496000000 else normal
attack if DstRate == 0 else normal
0.9758460116330558
attack if SrcAddr != 192.168.0.20 else normal
attack if SrcBytes == 124 else normal
attack if DstRate == 0 else normal
attack if DstPkts == 0 else normal
attack if DstLoad == 0 else normal
0.9713480968689643
attack if SrcAddr != 192.168.0.20 else normal
attack if TotPkts == 2 else normal
attack if DstRate == 0 else normal
attack if DstPkts == 0 else normal
attack if DstLoad == 0 else normal
0.9713480968689643


In [25]:
################################################################################
# Evaluate generated rules for efficiency
################################################################################

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from tabulate import tabulate
from statistics import mode
import time
import warnings
import pandas as pd
import os

warnings.filterwarnings("ignore")

sample_size = 100000

# Load dateset
df = pd.read_csv(os.getcwd() + f'/data/sample-{sample_size}-2.csv')

# Encode categorical columns
label_encoder = LabelEncoder()
categorical_columns = df.select_dtypes(include=['object']).columns
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])

# Split dataset according to attack type
normal_df = df[df['Target'] == 0]
attack_df = df[df['Target'] == 1]

# Split dataset into training and test set
normal_df_train = normal_df.sample(frac=0.8, random_state=42)
normal_df_test = normal_df.drop(normal_df_train.index)
attack_df_train = attack_df.sample(frac=0.8, random_state=42)
attack_df_test = attack_df.drop(attack_df_train.index)

X_train = pd.concat([normal_df_train, attack_df_train]).drop(columns=['Target', 'Traffic'])
y_train = pd.concat([normal_df_train, attack_df_train])['Target']
X_test = pd.concat([normal_df_test, attack_df_test]).drop(columns=['Target', 'Traffic'])
y_test = pd.concat([normal_df_test, attack_df_test])['Target']

# Create instances of ML models
model_dt = DecisionTreeClassifier()
model_rf = RandomForestClassifier()

# Fit the models to the training data
model_dt.fit(X_train, y_train)
model_rf.fit(X_train, y_train)

# Predict the labels for the test data
y_true = y_test

elapsed_times_dt = []
elapsed_times_rf = []
elapsed_times_llm = []
y_pred_dt = []
y_pred_rf = []
y_pred_llm = []
for i in range(len(X_test)):
    # Predict using DT
    start = time.time()
    y_pred_dt.append(model_dt.predict([X_test.iloc[i]]))
    end = time.time()
    elapsed_times_dt.append(end - start)

    # Predict using RF
    start = time.time()
    y_pred_rf.append(model_rf.predict([X_test.iloc[i]]))
    end = time.time()
    elapsed_times_rf.append(end - start)
    
    # Predict using LLM
    start = time.time()
    row = X_test.iloc[i]
    # conditions = [
    #     row['SrcAddr'] != "192.168.0.20",
    #     row['SrcRate'] == 1000000,
    #     row['DstRate'] == 0,
    #     row['DstPkts'] == 0,
    #     row['Rate'] == 1000000
    # ]
    conditions = [
        row['SrcAddr'] != "192.168.0.20",
        row['TotPkts'] == 2,
        row['DstRate'] == 0,
        row['DstPkts'] == 0,
        row['DstLoad'] == 0
    ]
    predicted_attack_types = ["attack" if condition else "normal" for condition in conditions]
    y_pred_llm.append(mode(predicted_attack_types))
    end = time.time()
    elapsed_times_llm.append(end - start)

print(f"DT time taken: {sum(elapsed_times_dt)/len(X_test)}")
print(classification_report(y_true, y_pred_dt, digits=4, output_dict=False))
print(confusion_matrix(y_true, y_pred_dt))
print("\n")

print(f"RF time taken: {sum(elapsed_times_rf)/len(X_test)}")
print(classification_report(y_true, y_pred_rf, digits=4, output_dict=False))
print(confusion_matrix(y_true, y_pred_rf))
print("\n")

print(f"LLM time taken: {sum(elapsed_times_llm)/len(X_test)}\n")
print(classification_report(["attack" if y else "normal" for y in y_true], y_pred_llm, digits=4, output_dict=False))
print(confusion_matrix(["attack" if y else "normal" for y in y_true], y_pred_llm))

DT time taken: 0.001401893651485443
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000     10000
           1     1.0000    1.0000    1.0000     10000

    accuracy                         1.0000     20000
   macro avg     1.0000    1.0000    1.0000     20000
weighted avg     1.0000    1.0000    1.0000     20000

[[10000     0]
 [    0 10000]]


RF time taken: 0.016973010945320128
              precision    recall  f1-score   support

           0     0.9999    0.9999    0.9999     10000
           1     0.9999    0.9999    0.9999     10000

    accuracy                         0.9999     20000
   macro avg     0.9999    0.9999    0.9999     20000
weighted avg     0.9999    0.9999    0.9999     20000

[[9999    1]
 [   1 9999]]


LLM time taken: 0.000884722650051117

              precision    recall  f1-score   support

      attack     0.9796    0.9632    0.9713     10000
      normal     0.9638    0.9799    0.9718     10000

    accura